In [1]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('Reiniciar kernel después de insttalar Apache Spark')
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_9603a7b4cc7a4df9b1d5fda9f44d74f9 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='qcBdsvg-rBsPYy813yg--cYEyEG0u-umrkMSlRYglVoK',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_9603a7b4cc7a4df9b1d5fda9f44d74f9.get_object(Bucket='prueba1-donotdelete-pr-x3bdtqvtvyhuvc',Key='Canada.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_0 = pd.read_excel(body)
df_data_0.head()


ModuleNotFoundError: No module named 'ibm_boto3'

### 1. Agregar las siguientes columnas al dataframe: migración 80s, migración 90s, migracion 2000s, migración 2010s y total migración

In [92]:
df_data_0["1980's"]=df_data_0.iloc[:,7:17].sum(axis=1)

In [93]:
df_data_0["1990's"]=df_data_0.iloc[:,17:27].sum(axis=1)

In [94]:
df_data_0["2000's"]=df_data_0.iloc[:,27:41].sum(axis=1)

### 2. Crear un nuevo df con la migración total por cada continente (solo 5 continentes) y convertirlo a Schema

In [95]:
 df=df_data_0.groupby("Continente",as_index=False).sum()

In [96]:
df=df[["Continente", "1980's", "1990's","2000's"]]
df.columns

Index(['Continente', '1980's', '1990's', '2000's'], dtype='object')

In [97]:
from pyspark.sql.types import *
mySchema = StructType([ StructField("Continente", StringType(), True)\
                       ,StructField("1980's", IntegerType(), True)\
                       ,StructField("1990's", IntegerType(), True)\
                       ,StructField("2000's", IntegerType(), True)\
                      ])

In [98]:
df2 = spark.createDataFrame(df,schema=mySchema)

In [99]:
df2.show()

+--------------------+------+------+-------+
|          Continente|1980's|1990's| 2000's|
+--------------------+------+------+-------+
|              Africa| 48815|148849| 421284|
|                Asia|351025|938744|2028025|
|              Europe|381738|548435| 480774|
|Latin America and...|185975|242802| 336371|
|    Northern America| 76824| 56931| 107387|
|             Oceania| 13096| 18430|  23648|
+--------------------+------+------+-------+



### 3. Crear un schema con los paises de oceania y las columnas de migración 80s, migración 90s, migracion 2000s, migración 2010s y total migración

In [100]:
df=df_data_0[df_data_0["Continente"]=="Oceania"][["Pais", "1980's", "1990's","2000's"]]
df["Total_Migracion"]=df.iloc[:,1::].sum(axis=1)


In [101]:
from pyspark.sql.types import *
mySchema = StructType([ StructField("Pais", StringType(), True)\
                       ,StructField("1980's", IntegerType(), True)\
                       ,StructField("1990's", IntegerType(), True)\
                       ,StructField("2000's", IntegerType(), True)\
                       ,StructField("Total_Migracion", IntegerType(), True)\
                      ])

In [102]:
df2 = spark.createDataFrame(df,schema=mySchema)

In [103]:
df2.show()

+----------------+------+------+------+---------------+
|            Pais|1980's|1990's|2000's|Total_Migracion|
+----------------+------+------+------+---------------+
|  American Samoa|     3|     2|     1|              6|
|       Australia|  4564|  6574| 12691|          23829|
|            Fiji|  5721|  9397|  5491|          20609|
|        Kiribati|     3|     5|     7|             15|
|Marshall Islands|     0|     0|     2|              2|
|           Nauru|     2|     1|    15|             18|
|   New Caledonia|     0|     0|     5|              5|
|     New Zealand|  2719|  2344|  5323|          10386|
|           Palau|     0|     0|     1|              1|
|Papua New Guinea|    17|    16|    31|             64|
|           Samoa|    23|    25|    30|             78|
|           Tonga|    41|    60|    41|            142|
|          Tuvalu|     3|     1|     4|              8|
|         Vanuatu|     0|     5|     6|             11|
+----------------+------+------+------+---------

### 4. Mostrar las estadisticas del Schema

In [104]:
df2.describe().show()

+-------+--------------+------------------+------------------+-----------------+-----------------+
|summary|          Pais|            1980's|            1990's|           2000's|  Total_Migracion|
+-------+--------------+------------------+------------------+-----------------+-----------------+
|  count|            14|                14|                14|               14|               14|
|   mean|          null| 935.4285714285714|1316.4285714285713|1689.142857142857|           3941.0|
| stddev|          null|1935.5988027674346| 2945.115636081224|3716.966709090023|8241.599583351988|
|    min|American Samoa|                 0|                 0|                1|                1|
|    max|       Vanuatu|              5721|              9397|            12691|            23829|
+-------+--------------+------------------+------------------+-----------------+-----------------+



### 5. Mostar las columnas del Schema en una lista

In [105]:
df2.columns

['Pais', "1980's", "1990's", "2000's", 'Total_Migracion']

### 6. Encontrar el promedio, la desviación estandard, el skewnesss y kurtosis de las columnas: migración 80s, migración 90s, migracion 2000s, migración 2010s y total migración

In [119]:
lista1=list(df["1980's"])
lista2=list(df["1990's"])
lista3=list(df["2000's"])
lista4=list(df['Total_Migracion'])
listas=[lista1,lista2,lista3]
lista_dec=["1980's","1990's","2000's",'Total_Migracion']

In [120]:
for i,j in zip(listas, lista_dec):
    rdd=sc.parallelize(i)
    suma = rdd.sum()
    promedio=suma/rdd.count()
    print(j," PROMEDIO: ",promedio)
    res1= rdd.map(lambda x:pow(x-promedio,2)).sum()/rdd.count()
    std = sqrt(res1)
    print(j," STD: ",std)
    res1= rdd.map(lambda x:pow(x-promedio,2)).sum()/rdd.count()
    n = float(rdd.count())
    skewness = 1/n*rdd.map(lambda x : pow(x-promedio,3)/pow(std,3)).sum()
    print(j," skewness: ",skewness)
    kurtosis = 1/n*rdd.map(lambda x : pow(x-promedio,4)/pow(std,4)).sum()
    print(j," kurtosis: ",kurtosis)


1980's  PROMEDIO:  935.4285714285714
1980's  STD:  1865.1896768458282
1980's  skewness:  1.6982694823341717
1980's  kurtosis:  4.226227843545893
1990's  PROMEDIO:  1316.4285714285713
1990's  STD:  2837.9844385530705
1990's  skewness:  2.029732501977548
1990's  kurtosis:  5.5724583580016915
2000's  PROMEDIO:  1689.142857142857
2000's  STD:  3581.758743114554
2000's  skewness:  2.14952699539004
2000's  kurtosis:  6.562484291933218


### 7. Obtener los valores maximos de las columnas

In [60]:
from pyspark.sql import *

In [121]:
maximo1=df2.agg({"1980's": "max"}).collect()[0][0]
maximo2=df2.agg({"1990's": "max"}).collect()[0][0]
maximo3=df2.agg({"2000's": "max"}).collect()[0][0]
print("1980's: ",maximo1)
print("1990's: ",maximo2)
print("2000's: ",maximo3)

1980's:  5721
1990's:  9397
2000's:  12691


### 8. Cambiar el nombre de los paises a mayusculas utilizando .replace()

In [122]:
pais=df2.select('Pais').rdd
pais=pais.map(lambda x: x[0].upper())#.take(20)
pais.collect()

['AMERICAN SAMOA',
 'AUSTRALIA',
 'FIJI',
 'KIRIBATI',
 'MARSHALL ISLANDS',
 'NAURU',
 'NEW CALEDONIA',
 'NEW ZEALAND',
 'PALAU',
 'PAPUA NEW GUINEA',
 'SAMOA',
 'TONGA',
 'TUVALU',
 'VANUATU']

### 9. Encontrar el pais con más migración en los 90s

In [123]:
maximo=df2.agg({"1990's": "max"}).collect()
df2.filter(df2["1990's"]==maximo[0][0]).show()

+----+------+------+------+---------------+
|Pais|1980's|1990's|2000's|Total_Migracion|
+----+------+------+------+---------------+
|Fiji|  5721|  9397|  5491|          20609|
+----+------+------+------+---------------+

